# Composition de musique note par note

## Introduction
Voici le notebook représentant ce que nous avons réalisé durant cette 2éme semaine de projet.

lorsque il y a ambiguité, entre le nom d'une variable et un nom communs, le nom de la variable est notés entre --, par exemple, 'les notes de la liste -notes-' ici le premier mot notes désigne les notes au sens commun (Do, Ré, Mi ...) tandis que -notes- désigne le nom de la variable python

### Imports

```python
import numpy as np
from __future__ import print_function
import tensorflow
from utils import *
import IPython
import sys
import numpy as np
from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K
```

## Description utils
utils est l'ensemble des fonctions permettant la relation entre les fichiers musiques en midi (via music21) et des matrices numpy permettant l'abstraction des dîtes musiques afin de faire passer l'analogie des musiques (matrices) au sein du réseau de neurones. (Bijection entre musique et matrices)


### Import de utils

```python
from keras.utils import to_categorical
from music21 import *
import numpy as np
```


### get_notes(path_to_midi)

#### argument(s) :
    - path_to_midi est une chaine de caractère representant le chemin de fichier midi
#### retour :
    get_notes(path_to_midi) renvoie une liste (-notes-) dont les éléments sont un à un les notes de la musique midi (ici notes et accords sont synonymes excepté que les notes sont représentées par un caractère et les accords par une suite de chiffres relié par des points

```python
def get_notes(path_to_midi):
    notes = []

    midi = converter.parse(path_to_midi)

    s2 = instrument.partitionByInstrument(midi)
    notes_to_parse = s2.parts[0].recurse()
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            # for n in element.pitches:
            #   print(str(n))
            notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

```

### generate_vocab(notes)

#### argument(s) :
    - notes est une liste dont les éléments sont des chaînes de caractères (lettre pour les notes, suite de chiffres séparés par des points pour les accord)
#### retour :
    generate_vocab(notes) renvoie l'ensemble du vocabulaire, c'est à dire la liste des éléments distincts de notes

```python
def generate_vocab(notes):
    """Generate vocabulary based on the input notes"""
    return np.unique(np.array(notes))
```

### generate_X_Y_from_one_music(note_to_index, notes, Tx, m)

#### argument(s) :
    - note_to_index est un dictionnaire reliant les notes aux indexes par rapport au vocabulaire 
    - notes est une liste dont les éléments sont des chaînes de caractères (lettre pour les notes, suite de chiffres séparés par des points pour les accord)
    - Tx est la longueur d'un training examples
    - m est le nombre de séquences souhaités pour une musique donnés
#### retour :
    generate_X_Y_from_one_music(name_to_index, notes, Tx, m) renvoie X et Y où X et Y sont des tenseurs à 3 dimensions comprenant l'ensemble des 'trainings examples', qui pour chaque 'training example' (ici ce nombre correspond au nombres de séquences de Tx notes, cette séquence est choisie aléatoirement dans la suite de notes -notes-) il y a une suite de Tx notes

```python
def generate_X_Y_from_one_music(name_to_index, notes, Tx, m):
    """Generate vectors X and Y for training where X[i+1]=Y[i]"""
    N_values = len(note_to_index)
    np.random.seed(0)
    X = np.zeros((m, Tx, N_values), dtype=np.bool)
    Y = np.zeros((m, Tx, N_values), dtype=np.bool)
    for i in range(m):
        random_idx = np.random.choice(len(notes) - Tx)
        notes_data = notes[random_idx:(random_idx + Tx)]
        for j in range(Tx):
            idx = note_to_index[notes_data[j]]
            if j != 0:
                X[i, j, idx] = 1
                Y[i, j - 1, idx] = 1

    Y = np.swapaxes(Y, 0, 1)
    Y = Y.tolist()
    return np.asarray(X), np.asarray(Y)
```

### generate_midi_file(outputName, notes)

#### argument(s) :
    - outputName est une chaîne de caractères (se terminant par .mid) désignant le nom du fichier midi généré  
    - notes est une liste dont les éléments sont des chaînes de caractères (lettre pour les notes, suite de chiffres séparés par des points pour les accord)
#### retour :
    generate_midi_file(outputName, notes) écrit le fichier midi 'outputName' en placant à la suites les notes de -notes-

```python
def generate_midi_file(outputName, notes):
    sheet = stream.Stream()
    for x in notes:
        if RepresentsInt(x[0]):
            ch = x.split(".")
            sheet.append(chord.Chord([int(k) for k in ch], quarterLength=0.25))
        else:
            sheet.append(note.Note(x, quarterLength=0.25))
    mf = midi.translate.streamToMidiFile(sheet)
    mf.open(outputName, "wb")
    mf.write()
    mf.close()
```





## Main
Voici donc le script principal ou sont défini les fonctions qui permettent au réseau de neurones de fonctionner et un exemple de ce réseau de neurones.

### Définitions des variables globales.

In [ ]:
notes = get_notes("elise.mid")
voc = generate_vocab(notes)
notes_to_ix = {n: i for i, n in enumerate(voc)}
ix_to_notes = {i: n for i, n in enumerate(voc)}
n_values = len(ix_to_notes)

X, Y = generate_X_Y_from_one_music(notes_to_ix, notes, 200, 10)

Tx = X.shape[1]

n_a = 64
reshapor = Reshape((1, n_values))
LSTM_cell = LSTM(n_a, return_state=True)
densor = Dense(n_values, activation='softmax')


### music_model(Tx, n_a, n_values)

#### argument(s) :
    - Tx est un entier correspondant à la longueur d'une séquence (c'est un nombre de notes) 
    - n_a donne le nombres de couches du RNN (le nombres d'activations)
    - n_values donne la taille du vocabulaire
#### retour :
    music_model(Tx, n_a, n_values) renvoie un model keras

In [ ]:
def music_model(Tx, n_a, n_values):
    X = Input(shape=(Tx, n_values))
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    outputs = []

    for t in range(Tx):
        x = Lambda(lambda x: X[:, t, :])(X)
        x = reshapor(x)
        a, _, c = LSTM_cell(x, initial_state=[a, c])
        out = densor(a)
        outputs.append(out)

    model = Model([X, a0, c0], outputs)

    return model

### Initialisation du model

In [ ]:
model = music_model(Tx=Tx, n_a=n_a, n_values=n_values)

opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

m = X.shape[0]
a0 = np.zeros((m, n_a))
c0 = np.zeros((m, n_a))
model.fit([X, a0, c0], list(Y), epochs=100)

### music_inference_model(LSTM_cell, densor, n_values, n_a, Ty)

#### argument(s) :
    - LSTM_cell : est la cellule LSTM du model défini plus haut.
    - densor :fonction qui applique une couche de Dense à la sortie d'une LSTM
    - n_values : taille du vocabulaire
    - n_a donne le nombres de couches du RNN (le nombres d'activations)
    - Ty est le nombre de notes qui devra être générée
#### retour :
    renvoie un model Keras prêt pour la génération

In [ ]:
def music_inference_model(LSTM_cell, densor, n_values, n_a, Ty):
    x0 = Input(shape=(1, n_values))
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    
    a = a0
    c = c0
    x = x0

    outputs = []

    for t in range(Ty):
        a, _, c = LSTM_cell(x, initial_state=[a, c])
        out = densor(a)
        outputs.append(out)

        x = RepeatVector(1)(out)
    inference_model = Model([x0, a0, c0], outputs)
    return inference_model


## Initialisation du modele d'inférence

In [ ]:
inference_model = music_inference_model(LSTM_cell, densor, n_values=n_values, n_a=n_a, Ty=300)

x_initializer = np.zeros((1, 1, n_values))
a_initializer = np.zeros((1, n_a))
c_initializer = np.zeros((1, n_a))

## Construire la prédiction pour la génération
### predict_and_sample(inference_model, x_initializer, a_initializer, c_initializer)

#### argument(s) :
    - inference_model : un model Keras qui permettra de "deviner quel sera la prochaine note à jouer
    - x_initializer : tableau numpy qui represente la premiere note jouée
    - a_initializer : est le tableau permettant aux activations de s'initialiser
    - c_initializer : est le tableau permettant d'initialiser
    - Ty est le nombre de notes qui devra être générée
#### retour :
    renvoie un couple, où
    - result : un tableau comprenant la prochaine note joué
    - indices : tableau comprenant les probabilités des a être joué.

In [ ]:
def predict_and_sample(inference_model, x_initializer, a_initializer, c_initializer):
    pred = inference_model.predict([x_initializer, a_initializer, c_initializer])
    indices = np.argmax(pred, axis=2)
    results = to_categorical(indices)
    return results, indices

## Génération de la musique

In [ ]:
generate_midi_file("eliseOver.mid", generate_music())